In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge,Lasso

In [ ]:
data=pd.read_csv('../input/car-price-prediction/CarPrice_Assignment.csv')

In [ ]:
data.head()

Drop unimportant column

In [ ]:
data=data.drop(['car_ID','symboling','CarName'],axis=1)

Check for missing and null values

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

## **Dealing with categorical variables**

In [ ]:
data_cat= data.select_dtypes(include='object')

In [ ]:
data_cat

In [ ]:
data_cat_dummy=pd.get_dummies(data_cat,drop_first=True)

In [ ]:
data_cat_dummy

In [ ]:
data_num=data.select_dtypes(exclude='object')

In [ ]:
data_final= data_num.join(data_cat_dummy)
data_final

In [ ]:
X= data_final.drop('price',axis=1)
y=data_final['price']

In [ ]:
for i in X.columns:
    plt.scatter(X[i],y)
    plt.title("price and "+ i)
    plt.show()

### Selecting relevant columns with feature selection

In [ ]:
from sklearn.feature_selection import f_regression

In [ ]:
summary = pd.DataFrame(data=X.columns,columns=['Features'])
summary['p_values']=f_regression(X,y)[1].round(3)

In [ ]:
X=X.drop(summary[summary['p_values']>0.005]['Features'],axis=1)

#### Relevant features are those with p-value = 0.000 which includes 'enginesize'

In [ ]:
data_final.corr()['price'][['enginesize','peakrpm','carbody_sedan','carbody_hatchback']]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=25)

In [ ]:
reg=LinearRegression()
reg.fit(x_train,y_train)
reg.coef_

In [ ]:
regsummary=pd.DataFrame(data=X.columns,columns=['features'])

### From the regression coefficient and plots
#### price increases with engine size
#### car(hatchback) causes the price to reduce
### From feature selection and plots
#### car(sedan) has no effect on the price
#### peakrpm has no effect on the price

In [ ]:
reg.score(x_test,y_test)

### Linear Model has an R squared value of 0.85

In [ ]:
alpha=[10,1,0.1,0.0001,1e-8]
rid_score=[]
las_score=[]
for i in alpha:
    rid=Ridge(alpha=i)
    las=Lasso(alpha=i)
    rid.fit(x_train,y_train)
    las.fit(x_train,y_train)
    rid_score.append(rid.score(x_train,y_train))
    las_score.append(las.score(x_train,y_train))

In [ ]:
rid_score,np.mean(rid_score)

In [ ]:
las_score,np.mean(las_score)

### Ridge regression has a mean R squared of 0.90
### Lasso regression has a mean R squared of 0.907